# Data Exploration

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'http://www.snlarchives.net/Episodes/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

In [ ]:
df = pd.read_html(url)

In [ ]:
# tables of episodes with host and musical guest
df[0]

In [ ]:
#generating a list of episode ids

df[46]
# each table is a season. table number - 1 = season number

In [ ]:
df_s47_test = df[46]
df_s47_test['sid'] = df_s47_test[0].apply(lambda x: '.'.join(x.split('.')[2::-1]))


In [ ]:
df_s47_test['sid'] = df_s47_test['sid'].str.replace('.', '')
df_s47_test

In [ ]:
df_s47_test.rename(columns={'sid': 'epid'}, inplace=True)
df_s47_test['sid'] = 47
df_s47_test.rename(columns={0: 'aired'}, inplace=True)
df_s47_test

In [ ]:
df_s47_test[['host', 'musical_guest']] = df_s47_test[1].str.split('/', expand=True)
df_s47_test['musical_guest'].fillna(df_s47_test['host'], inplace=True)
df_s47_test.drop(columns=1, inplace=True)
df_s47_test


In [ ]:
updated_tables = df_s47_test
updated_tables

In [ ]:
season = 48
temp_table = pd.DataFrame()
for table in df[47:]:
    print(table)
    # temp_table = table
    # temp_table['epid'] = temp_table[0].apply(lambda x: '.'.join(x.split('.')[2::-1]))
    # temp_table['epid'] = temp_table['epid'].str.replace('.', '')
    # temp_table['sid'] = season
    # temp_table.rename(columns={0: 'aired'}, inplace=True)
    # temp_table[['host', 'musical_guest']] = temp_table[1].str.split('/', expand=True)
    # temp_table['musical_guest'].fillna(temp_table['host'], inplace=True)
    # temp_table.drop(columns=1, inplace=True)
    # updated_tables = pd.concat([updated_tables, temp_table])
    # season += 1

updated_tables

In [ ]:
table

In [ ]:
updated_tables

In [ ]:
updated_tables['epno'] = updated_tables.index + 1
updated_tables


In [ ]:
updated_tables.to_csv('./data/updated_tables.csv', index=False)
print('done')


# Pulling the ratings from IMDB

In [ ]:
from playwright.async_api import async_playwright

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

In [ ]:
# testing with the first link
base_url = 'https://www.imdb.com/title/tt0072562/episodes/?season=49'
await page.goto(base_url)

In [ ]:
html = await page.content()
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

In [ ]:
for rating in soup.select('.sc-f2169d65-12 .ipc-rating-star'):
    # print(rating.get('aria-label'))
    print(rating.text)
    print('----')

In [ ]:
for title in soup.select(".sc-f2169d65-5 .ipc-title__text"):
    print(title)

In [ ]:
for date in soup.select('.sc-f2169d65-10'):
    print(date.text)

In [ ]:
title = []

In [ ]:
#let's put it all together:
all_data = []
ratings = []
titles = []
dates = []

for rating in soup.select('.sc-f2169d65-12 .ipc-rating-star'):
    ratings.append(rating.text)

for rating in ratings:
    if rating == 'Rate':
        ratings.remove(rating)

print(ratings)


for title in soup.select(".sc-f2169d65-4 .ipc-title__text"):
    titles.append(title.text)

print(titles)

for date in soup.select('.sc-f2169d65-10'):
    dates.append(date.text)

print(dates)




In [ ]:
for rating, title, date in zip(ratings, titles, dates):
    episode = {}
    # Perform operations on each element
    # For example, print the values
    # print(rating, title, date)
    episode['rating'] = rating
    episode['title'] = title
    episode['date'] = date
    all_data.append(episode)
    print(episode)

all_data

In [ ]:
df = pd.DataFrame(all_data)

In [ ]:
df

In [ ]:
df['ratings'] = df['rating'].str.extract('(.*?)\/')
df['votes'] = df['rating'].str.extract('\((.*?)\)')
df

In [ ]:
df['sid'] = df['title'].str.extract('^S(\d{2})')
df


In [ ]:
df['hosts'] = df['title'].str.extract('∙(.*)')
df


In [ ]:
# Convert the date string to datetime
df['date_datetime'] = pd.to_datetime(df['date'])

# Format the datetime into a purely digit string (YYYYMMDD)
df['epid'] = df['date_datetime'].dt.strftime('%Y%m%d')

df

# The actual imdb scraper

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from playwright.async_api import async_playwright
import asyncio

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

In [ ]:
done = False
season = 1
all_data = []
while(done == False):
    base_url = 'https://www.imdb.com/title/tt0072562/episodes/?season='
    url = base_url + str(season)

    # print(url)
    await page.goto(url)

    html = await page.content()
    soup = BeautifulSoup(html, 'html.parser')

    ratings = []
    titles = []
    dates = []

    for rating in soup.select('.sc-f2169d65-12 .ipc-rating-star'):
        ratings.append(rating.text)

    for rating in ratings:
        if rating == 'Rate':
            ratings.remove(rating)
    
    for title in soup.select(".sc-f2169d65-4 .ipc-title__text"):
        titles.append(title.text)
    
    for date in soup.select('.sc-f2169d65-10'):
        dates.append(date.text)

    all_season = []

    for rating, title, date in zip(ratings, titles, dates):
        episode = {}
        episode['rating'] = rating
        episode['title'] = title
        episode['date'] = date
        all_season.append(episode)
    
    all_data.append(all_season)

    print(f"season {season} done")

    season += 1

    if(season == 47):
        done = True

    await asyncio.sleep(4)


In [6]:
# writing it in bs4 T_T

done = False
season = 1
# all_data = []
while(done == False):
    base_url = 'https://www.imdb.com/title/tt0072562/episodes/?season='
    url = base_url + str(season)

    # print(url)
    # await page.goto(url)

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    html = requests.get(url, headers=headers)

    # html = await page.content()
    soup = BeautifulSoup(html.text, 'html.parser')

    ratings = []
    titles = []
    dates = []

    for rating in soup.select('.sc-f2169d65-12 .ipc-rating-star'):
        ratings.append(rating.text)

    for rating in ratings:
        if rating == 'Rate':
            ratings.remove(rating)
    
    for title in soup.select(".sc-f2169d65-4 .ipc-title__text"):
        titles.append(title.text)
    
    for date in soup.select('.sc-f2169d65-10'):
        dates.append(date.text)

    all_season = []

    for rating, title, date in zip(ratings, titles, dates):
        episode = {}
        episode['rating'] = rating
        episode['title'] = title
        episode['date'] = date
        all_season.append(episode)
    
    all_data.append(all_season)

    print(f"season {season} done")

    season += 1

    if(season == 10):
        done = True

    await asyncio.sleep(4)


season 1 done
season 2 done
season 3 done
season 4 done
season 5 done
season 6 done
season 7 done
season 8 done
season 9 done


In [4]:
# bs4 continued

done = False
season = 10
all_data = []
while(done == False):
    base_url = 'https://www.imdb.com/title/tt0072562/episodes/?season='
    url = base_url + str(season)

    # print(url)
    # await page.goto(url)

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    html = requests.get(url, headers=headers)

    # html = await page.content()
    soup = BeautifulSoup(html.text, 'html.parser')

    ratings = []
    titles = []
    dates = []

    for rating in soup.select('.sc-f2169d65-12 .ipc-rating-star'):
        ratings.append(rating.text)

    for rating in ratings:
        if rating == 'Rate':
            ratings.remove(rating)
    
    for title in soup.select(".sc-f2169d65-4 .ipc-title__text"):
        titles.append(title.text)
    
    for date in soup.select('.sc-f2169d65-10'):
        dates.append(date.text)

    all_season = []

    for rating, title, date in zip(ratings, titles, dates):
        episode = {}
        episode['rating'] = rating
        episode['title'] = title
        episode['date'] = date
        all_season.append(episode)
    
    all_data.append(all_season)

    print(f"season {season} done")

    season += 1

    if(season == 47):
        done = True

    await asyncio.sleep(4)


season 10 done
season 11 done
season 12 done
season 13 done
season 14 done
season 15 done
season 16 done
season 17 done
season 18 done
season 19 done
season 20 done
season 21 done
season 22 done
season 23 done
season 24 done
season 25 done
season 26 done
season 27 done
season 28 done
season 29 done
season 30 done
season 31 done
season 32 done
season 33 done
season 34 done
season 35 done
season 36 done
season 37 done
season 38 done
season 39 done
season 40 done
season 41 done
season 42 done
season 43 done
season 44 done
season 45 done
season 46 done


In [9]:
len(all_data)

46

In [10]:
all_data

[[{'rating': '6.2/10\xa0(89)',
   'title': 'S10.E1 ∙ Thompson Twins',
   'date': 'Sat, Oct 6, 1984'},
  {'rating': '6.0/10\xa0(82)',
   'title': 'S10.E2 ∙ Bob Uecker/Peter Wolf',
   'date': 'Sat, Oct 13, 1984'},
  {'rating': '6.0/10\xa0(91)',
   'title': 'S10.E3 ∙ Jesse Jackson/Andrae Crouch, Wintley Phipps',
   'date': 'Sat, Oct 20, 1984'},
  {'rating': '6.8/10\xa0(84)',
   'title': 'S10.E4 ∙ Michael McKean/Chaka Khan/The Folksmen',
   'date': 'Sat, Nov 3, 1984'},
  {'rating': '7.4/10\xa0(93)',
   'title': 'S10.E5 ∙ George Carlin/Frankie Goes to Hollywood',
   'date': 'Sat, Nov 10, 1984'},
  {'rating': '7.0/10\xa0(81)',
   'title': 'S10.E6 ∙ Ed Asner/The Kinks',
   'date': 'Sat, Nov 17, 1984'},
  {'rating': '5.5/10\xa0(86)',
   'title': 'S10.E7 ∙ Ed Begley Jr/Billy Squier',
   'date': 'Sat, Dec 1, 1984'},
  {'rating': '6.4/10\xa0(86)',
   'title': 'S10.E8 ∙ Ringo Starr/Herbie Hancock',
   'date': 'Sat, Dec 8, 1984'},
  {'rating': '7.2/10\xa0(108)',
   'title': 'S10.E9 ∙ Eddie Murphy/R

In [11]:
flattened_list = [item for sublist in all_data for item in sublist]
flattened_list

df_all_seasons = pd.DataFrame(flattened_list)

In [12]:
df_all_seasons

,rating,title,date
0,6.2/10 (89),S10.E1 ∙ Thompson Twins,"Sat, Oct 6, 1984"
1,6.0/10 (82),S10.E2 ∙ Bob Uecker/Peter Wolf,"Sat, Oct 13, 1984"
2,6.0/10 (91),"S10.E3 ∙ Jesse Jackson/Andrae Crouch, Wintley ...","Sat, Oct 20, 1984"
3,6.8/10 (84),S10.E4 ∙ Michael McKean/Chaka Khan/The Folksmen,"Sat, Nov 3, 1984"
4,7.4/10 (93),S10.E5 ∙ George Carlin/Frankie Goes to Hollywood,"Sat, Nov 10, 1984"
...,...,...,...
906,5.9/10 (91),S9.E15 ∙ Billy Crystal/Al Jarreau,"Sat, Mar 17, 1984"
907,6.2/10 (92),S9.E16 ∙ Michael Douglas/Deniece Williams,"Sat, Apr 7, 1984"
908,5.7/10 (90),S9.E17 ∙ George McGovern/Madness,"Sat, Apr 14, 1984"
909,7.2/10 (89),S9.E18 ∙ Barry Bostwick/Spinal Tap,"Sat, May 5, 1984"


In [13]:
#cleaning the data
df_all_seasons['ratings'] = df_all_seasons['rating'].str.extract('(.*?)\/')
df_all_seasons['votes'] = df_all_seasons['rating'].str.extract('\((.*?)\)')
df_all_seasons['sid'] = df_all_seasons['title'].str.extract('^S(\d+)')
df_all_seasons['hosts'] = df_all_seasons['title'].str.extract('∙(.*)')
df_all_seasons['date_datetime'] = pd.to_datetime(df_all_seasons['date'])
df_all_seasons['epid'] = df_all_seasons['date_datetime'].dt.strftime('%Y%m%d')
df_all_seasons


,rating,title,date,ratings,votes,sid,hosts,date_datetime,epid
0,6.2/10 (89),S10.E1 ∙ Thompson Twins,"Sat, Oct 6, 1984",6.2,89,10,Thompson Twins,1984-10-06,19841006
1,6.0/10 (82),S10.E2 ∙ Bob Uecker/Peter Wolf,"Sat, Oct 13, 1984",6.0,82,10,Bob Uecker/Peter Wolf,1984-10-13,19841013
2,6.0/10 (91),"S10.E3 ∙ Jesse Jackson/Andrae Crouch, Wintley ...","Sat, Oct 20, 1984",6.0,91,10,"Jesse Jackson/Andrae Crouch, Wintley Phipps",1984-10-20,19841020
3,6.8/10 (84),S10.E4 ∙ Michael McKean/Chaka Khan/The Folksmen,"Sat, Nov 3, 1984",6.8,84,10,Michael McKean/Chaka Khan/The Folksmen,1984-11-03,19841103
4,7.4/10 (93),S10.E5 ∙ George Carlin/Frankie Goes to Hollywood,"Sat, Nov 10, 1984",7.4,93,10,George Carlin/Frankie Goes to Hollywood,1984-11-10,19841110
...,...,...,...,...,...,...,...,...,...
906,5.9/10 (91),S9.E15 ∙ Billy Crystal/Al Jarreau,"Sat, Mar 17, 1984",5.9,91,9,Billy Crystal/Al Jarreau,1984-03-17,19840317
907,6.2/10 (92),S9.E16 ∙ Michael Douglas/Deniece Williams,"Sat, Apr 7, 1984",6.2,92,9,Michael Douglas/Deniece Williams,1984-04-07,19840407
908,5.7/10 (90),S9.E17 ∙ George McGovern/Madness,"Sat, Apr 14, 1984",5.7,90,9,George McGovern/Madness,1984-04-14,19840414
909,7.2/10 (89),S9.E18 ∙ Barry Bostwick/Spinal Tap,"Sat, May 5, 1984",7.2,89,9,Barry Bostwick/Spinal Tap,1984-05-05,19840505


In [18]:
df_all_seasons['sid'] = df_all_seasons['sid'].astype(int)


In [19]:
df_all_seasons.sort_values('sid', inplace=True)
df_all_seasons


,rating,title,date,ratings,votes,sid,hosts,date_datetime,epid
732,7.4/10 (543),S1.E1 ∙ Saturday Night Live: George Carlin/Bil...,"Sat, Oct 11, 1975",7.4,543,1,Saturday Night Live: George Carlin/Billy Pres...,1975-10-11,19751011
733,6.2/10 (354),S1.E2 ∙ Paul Simon/Randy Newman/Phoebe Snow,"Sat, Oct 18, 1975",6.2,354,1,Paul Simon/Randy Newman/Phoebe Snow,1975-10-18,19751018
734,6.9/10 (303),S1.E3 ∙ Rob Reiner,"Sat, Oct 25, 1975",6.9,303,1,Rob Reiner,1975-10-25,19751025
735,7.0/10 (281),S1.E4 ∙ Candice Bergen/Esther Phillips,"Sat, Nov 8, 1975",7.0,281,1,Candice Bergen/Esther Phillips,1975-11-08,19751108
736,6.5/10 (245),"S1.E5 ∙ Robert Klein/ABBA, Loudon Wainwright III","Sat, Nov 15, 1975",6.5,245,1,"Robert Klein/ABBA, Loudon Wainwright III",1975-11-15,19751115
...,...,...,...,...,...,...,...,...,...
713,6.8/10 (532),S46.E2 ∙ Bill Burr/Jack White,"Sat, Oct 10, 2020",6.8,532,46,Bill Burr/Jack White,2020-10-10,20201010
726,6.1/10 (332),S46.E15 ∙ Maya Rudolph/Jack Harlow,"Sat, Mar 27, 2021",6.1,332,46,Maya Rudolph/Jack Harlow,2021-03-27,20210327
712,6.4/10 (501),S46.E1 ∙ Chris Rock/Megan Thee Stallion,"Sat, Oct 3, 2020",6.4,501,46,Chris Rock/Megan Thee Stallion,2020-10-03,20201003
720,6.8/10 (368),S46.E9 ∙ Kristen Wiig/Dua Lipa,"Sat, Dec 19, 2020",6.8,368,46,Kristen Wiig/Dua Lipa,2020-12-19,20201219


In [23]:
df_all_seasons_cleaned = df_all_seasons.drop(columns=['title', 'date', 'rating'])
df_all_seasons_cleaned


KeyError: "['title', 'date', 'rating'] not found in axis"

In [22]:
df_all_seasons

,ratings,votes,sid,hosts,date_datetime,epid
732,7.4,543,1,Saturday Night Live: George Carlin/Billy Pres...,1975-10-11,19751011
733,6.2,354,1,Paul Simon/Randy Newman/Phoebe Snow,1975-10-18,19751018
734,6.9,303,1,Rob Reiner,1975-10-25,19751025
735,7.0,281,1,Candice Bergen/Esther Phillips,1975-11-08,19751108
736,6.5,245,1,"Robert Klein/ABBA, Loudon Wainwright III",1975-11-15,19751115
...,...,...,...,...,...,...
713,6.8,532,46,Bill Burr/Jack White,2020-10-10,20201010
726,6.1,332,46,Maya Rudolph/Jack Harlow,2021-03-27,20210327
712,6.4,501,46,Chris Rock/Megan Thee Stallion,2020-10-03,20201003
720,6.8,368,46,Kristen Wiig/Dua Lipa,2020-12-19,20201219


In [24]:
df_all_seasons.to_csv('df_all_seasons.csv', index=False)
print('done')

done
